<h1>Toronto</h1>

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

from bs4 import BeautifulSoup
import urllib.request

In [2]:
#Scrap postal codes V1
wikipedia='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webpage = urllib.request.urlopen(wikipedia)
soup = BeautifulSoup(webpage, 'html.parser')
table = soup.find_all("table", class_="wikitable sortable")
table

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B
 </td>
 <td>Not assigned
 </td>
 <td

In [3]:
#Scrap postal codes V2
postal_codes=pd.read_html(wikipedia, attrs={"class": "wikitable"})[0]

#Suppress not assigned borough
postal_codes=postal_codes[postal_codes.Borough!='Not assigned']

#Replace "Not assigned" Neighborough py borough
postal_codes['Neighborhood'].loc[postal_codes['Neighborhood'] =='Not assigned'] =postal_codes['Borough']

C:\Users\legea\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
postal_codes[postal_codes['Postal Code']=="M5A"]

,Postal Code,Borough,Neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
print(postal_codes.shape)

(103, 3)


In [8]:
#Tried to import data with geocoder, but it takes too long.
import geocoder
latitudes=[]
longitudes=[]
for i in postal_codes['Postal Code']:
    print('{}, Toronto, Ontario'.format(i))
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(i))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    latitudes.append(latitude)
    longitude = lat_lng_coords[1]
    longitudes.append(longitude)

M3A, Toronto, Ontario


KeyboardInterrupt: 

In [11]:
#Import data from csv
latlong=pd.read_csv('https://cocl.us/Geospatial_data')

#Merge with postal_codes

pc=postal_codes.merge(latlong,on='Postal Code')

In [16]:
if pc.shape[0]==postal_codes.shape[0]:
    print("Same number of rows, merge is ok")
else:
    print("Initial was {} rows, merged is {} rows, not ok".format(postal_codes.shape[0],pc.shape[0]))

pc.head()

Same number of rows, merge is ok


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
